# Carlane: Experiment Code

train and test mask rcnn model of Carlane dataset

In [ ]:
import os
import numpy as np
import sys
ROOT_DIR = os.path.abspath("../code")
sys.path.append(ROOT_DIR)
import carlane

from mrcnn import model as modellib, utils
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

In [ ]:
logs = '../log'
if not os.path.exists(logs):
    os.mkdir(logs)
    
command = "train" #  or val
dataset_dir = "../data/2.7万张道路线精细标注抽取2000张"

weights = "imagenet" # or coco, or last, or 

In [ ]:
if command == "train":
        config = carlane.CarlaneConfig()
else:
    class InferenceConfig(carlane.CarlaneConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
    config = InferenceConfig()
config.display()


In [ ]:
if command == "train":
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=logs)
else:
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=logs)

In [ ]:
if weights.lower() == "imagenet":
    # Start from ImageNet trained weights
    weights_path = model.get_imagenet_weights()
else:
    weights_path = weights

In [ ]:
model.load_weights(weights_path, by_name=True)

In [ ]:
# Training dataset.
dataset_train = carlane.CarlaneDataset()
dataset_train.load_carlane(dataset_dir, "train")
dataset_train.prepare()

# Validation dataset
dataset_val = carlane.CarlaneDataset()
dataset_val.load_carlane(dataset_dir, "val")
dataset_val.prepare()


In [ ]:
print("train images count:{}, val images count: {}".format(dataset_train.num_images, dataset_val.num_images))

In [ ]:
import imgaug
# Image Augmentation
# Right/Left flip 50% of the time
augmentation = imgaug.augmenters.Fliplr(0.5)

# *** This training schedule is an example. Update to your needs ***

# Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='heads',
            augmentation=augmentation)

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=120,
            layers='4+',
            augmentation=augmentation)

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=160,
            layers='all',
            augmentation=augmentation)

# Draft

In [4]:
import tensorflow as tf
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))


/home/user2/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


[[ 22.  28.]
 [ 49.  64.]]
[[ 22.  28.]
 [ 49.  64.]]
